In this notebook, we apply MCMC to a simple deciphering task. The problem is interesting, because the state space of the markov chain is permutations of the alphabet, so there are (26 + 1)! $\approx 10^{28}$ states. The main point of this is to show how MCMC can easily sample from a bizarre distribution defined over a huge state space. The MCMC algorithm itself is quite simple and converges surprisingly quickly. 

The goal is to crack a substitution cipher. That is, a cipher where every letter of the alphabet has been switched for some symbol (we'll just use the letter of the alphabet again as our symbols). The way we intend to solve this is to set up a Markov chain where the stationary distribution is concentrated, in some sense, on the "most plausible" deciphering schemes. We'll do this as follows.  

Firsts, compute a bigram model of English using some source text that you think statistically resembles the coded text. We'll use war and peace. Let $b(x, y)$ denote the frequency of the bigram $(x,y)$ in your source text. Let $f$ be a deciphering scheme (mapping from coded symbols to english letters). We'll define the plausiblity of $f$ to be
$$
Pl(f) := \prod_{c_i} b(f(c_i), f(c_{i+1})),
$$
where $c_i$ denotes the $i$th character in your coded message. Basically, if you decode your message with $f$ and it gives a bunch of unlikely bigrams (meaning they rarely occur in your reference text), then it's implausible. On the other hand, if many of the bigrams that appear in the decoded message are also common in your reference text, then you consider it to be more plausible. 

Now, observe that the set of possible $f$'s is essentially the set of permutations of the alphabet. (e.g., if we're going from regular alphabet symbols to regular alphabet symbols, then then your cipher is just a permutation map that tells you which letter in a regular message maps to which coded letter). We'll use lowercase letters and include a space character, so we have $27!$ states. Let
$$
\pi(f; b, \text{message}) := \frac{Pl(f)}{\sum_g Pl(g)}
$$
denote a normalization of $Pl(f)$, where the sum is over all possible permutations $g$. I added the parameters $b$ and the coded message as "parameters" to $\pi$ to emphasize that distribution depends critically on both of these. If you change your bigram model or your decoded message, then you're going to shift $\pi$. 

We want to sample from $\pi$. The idea is that most of the mass of $\pi$ should be on reasonable decoding schemes. So, a sample from it should give a decent first guess at a decoding scheme. 

To sample from $\pi$ we'll use a very simple MCMC algorithm that will be described below. The algorithm is really strikingly simple. 

We'll start by building a bigram model. We'll use a copy of war and peace downloaded from project gutenberg as a reference text. Nothing too interesting here. 

In [1]:
import numpy as np
import random
import pickle

recompute_bigram = True
smoothing_k = 1  # smoothing factor for bigram model (pretend there are this many examples to start with for each bigram)
if recompute_bigram:
    # read in war and peace
    words = open('data/war_and_peace.txt', 'r').read().lower().split()

    # compute bigram model
    b = {}
    alphabet = set(' .!abcdefghijklmnopqrstuvwxyz')  # restrict alphabet to 26 + 1 characters
    for w in words:
        chs = [' '] + list(w) + [' ']
        for ch1, ch2, ch3 in zip(chs, chs[1:], ch[2:]):

            # Get rid of weird characters
            if ch1 not in alphabet or ch2 not in alphabet or ch3 not in alphabet:
                continue

            # track frequency
            bigram = (ch1, ch2, ch3)
            b[bigram] = b.get(bigram, 0) + 1

    # normalize bigram model
    total = sum((v for v in b.values())) + smoothing_k*len(alphabet)  # + stuff for smoothing
    for k, v in b.items():
        b[k] = v/total

    stuff = (alphabet, b)
    pickle.dump(stuff, open('bigram_model.p', 'wb'))

else:
    alphabet, b = pickle.load(open('bigram_model.p', 'rb'))


NameError: name 'ch' is not defined

Next, let's pick a message we want to code. 

In [ ]:
input_text =  "hello brian! i am glad you figured out the code. with a little help from your computer of course. i had to think for a second to make this longer but this is it. hopefully the computer can guess it."

And now let's generate a random cipher. 

In [ ]:
# # come up with a cipher
sorted_alphabet = sorted(alphabet)
# sigma = np.random.permutation(sorted_alphabet)
# print(sigma)
# cipher = {}
# for i, ch in enumerate(sorted_alphabet):
#     cipher[ch] = sigma[i]

cipher = {'a': ".",
          'b': "f",
          'c': "z",
          'd': "x",
          'e': "!",
          'f': "q",
          'g': "b",
          'h': "g",
          'i': " ",
          'j': "y",
          'k': "w",
          'l': "o",
          'm': "i",
          'n': "c",
          'o': "a",
          'p': "p",
          'q': "t",
          'r': "h",
          's': "d",
          't': "s",
          'u': "l",
          'v': "m",
          'w': "n",
          'x': "u",
          'y': "v",
          'z': "j",
          ' ': "e",
          '!': "r",
          '.': "k",
          }

print(f'cipher={cipher}')
    
# compute inverse cipher. Mostly for debugging...
real_inv_cipher = {}
for k, v in cipher.items():
    real_inv_cipher[v] = k

# cipher some input text
cipher_text = ''.join([cipher[ch] for ch in input_text])
print(f'ciphertext = {cipher_text}')

Before running the MCMC algorithm, here are a couple of simple helper functions. 

In [ ]:
def decipher_text(cipher_candidate):
    "Using the cipher candidate, decode the cipher_text above."
    deciphered_text = []
    for ch in cipher_text:
        deciphered_text.append(cipher_candidate[ch])
    return deciphered_text

def plaus(cipher_candidate):
    "Compute plausiblity, Pl(cipher_candidate)."
    deciphered_text = decipher_text(cipher_candidate)
    score = 0
    for ch1, ch2 in zip(deciphered_text, deciphered_text[1:]):
        score += np.log(b.get((ch1, ch2), smoothing_k/total))
    return score

As a sanity check, let's apply the real decipher key and make sure it works. 

In [ ]:
# sanity check that the real decipher key works
deciphered_text = "".join(decipher_text(real_inv_cipher))
print(f'deciphered text: {deciphered_text}')

Now we're ready for MCMC. 

Pseudo code for the algorithm is as follows (taken from Diaconis):
1. Start with a random permutation $f$.
2. compute $Pl(f)$.
3. Change to $f^*$ by making a random transposition of the values $f$ assigns to two symbols.
4. Compute $Pl(f^*)$; if this is larger than $Pl(f)$, accept $f^*$.
5. If not, flip a $Pl(f^*)/Pl(f)$ coin; if it comes up heads, accept $f^*$.
6. If the coin comes up tails, stay at $f$.

This is an instance of the metropolis algorithm. Which is a special subclass of metropolis hasings algorithms where the proposal distribution is symmetric. This is why the accept rule only has the target distribution in it (the usual proposal distribution terms from metropolis hasings cancel out due to symmetry.) 

The metropolis algorithm is discussed in Algorithm A.29, p.288 in MCSM. 



In [ ]:
print(f'length of ciphertext={len(cipher_text)}')
# for later reference, print out the plausiblity score of the real deciphering scheme
print(f'REAL CIPHER PLAUSIBLITY: {plaus(real_inv_cipher)}')

# initialize random (inverse) cipher guess
sigma = np.random.permutation(sorted_alphabet)
inv_cipher = {}
for i, ch in enumerate(sorted_alphabet):
    inv_cipher[ch] = sigma[i]

# Run metropolis algorithm
n_steps = 20_000
for t in range(n_steps):
    # Generate permutation candidate (this should be a uniform random sample from the set of permutations, right?)
    new_cipher = inv_cipher.copy()
    ch1, ch2 = random.sample(sorted_alphabet, 2)
    temp_val = new_cipher[ch1]
    new_cipher[ch1] = new_cipher[ch2]
    new_cipher[ch2] = temp_val

    # metropolis update
    if plaus(new_cipher) > plaus(inv_cipher):
        inv_cipher = new_cipher
    else:
        threshold = np.exp(plaus(new_cipher) - plaus(inv_cipher))
        if np.random.uniform() < threshold:
            inv_cipher = new_cipher

    if t % 100 == 0:
        deciphered_text = "".join(decipher_text(inv_cipher))[:80]
        print(f't={t}; score = {plaus(inv_cipher):.2f}; text={deciphered_text}...')
        #print(f'score = {plaus(inv_cipher)}, real cipher score = {plaus(real_inv_cipher)}', )
#         print(f'{deciphered_text}')

In [ ]:
inv_cipher[' '] = 'i'
inv_cipher['.'] = 'a'

In [ ]:
print(inv_cipher)
print(cipher_text)
deciphered_text = "".join(decipher_text(inv_cipher))
print(f't={t}; score = {plaus(inv_cipher):.2f}; text={deciphered_text}')

Note: Score is $ln(Pl(f))$

Closing thoughts:


- The point of this example isn't that this is a useful decryption scheme. It's that we've been able to sample from this really weird and custom distribution. We've defined a distribution over a state space of $~10^{28}$ permutations. The exact distribution we need to sample from changes whenever we change the message or the bigram reference. Yet, we seem to get reasonable mixing within a few thousand iterations without any special initialization.
- It is sensitive to initialization. Some initializations don't work well at all. But you don't to try too hard. A uniform random one will get you there without too much work. 
- The choice of proposal distribution is critical. If you mess with the way new permutations are proposed or the accept threshold, it can easily go haywire. 
- The fact that we're trying to construct a distribution that concentrates on the set of maxima of some function begs the idea that we consider simulated annealing. This is accomplished with a really simple modification of this code. Just add a temperature parameter to the cost function in the form of an exponent. As the exponent goes $\infty$ (so, use $1/T$ as the exponent, where $T$ is the temperature), $\pi$ concentrates on the set of maxima of $Pl(f)$. 


Some useful references:
- "The markov chain monte carlo revolution", Perci Diaconis
- "Decrypting classical cipher text using Markov chain Monte Carlo", Jian Chen, Jeffrey S. Rosenthal
- (MCSM) Monte Carlo statistical methods, book by Roberts, and others